In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv("../../keras/CONLL_dataset.csv")

In [2]:


tags = np.unique(df.Tag)

tags

transform_tag_mapping = {
    'B-art': ['Object'],
    'B-eve': ['Event'],
    'B-geo': ['Location', 'Party'],
    'B-gpe': ['Race'],
    'B-nat': ['SpecialTerm'],
    'B-org': ['Party'],
    'B-per': ['Party'],
    'B-tim': ['Time'],
    'I-art': ['Object'],
    'I-eve': ['Event'],
    'I-geo': ['Location', 'Party'],
    'I-gpe': ['Race', "Party"],
    'I-nat': ['SpecialTerm'],
    'I-org': ['Party'],
    'I-per': ['Party'],
    'I-tim': ['TemporalUnit'],
    'O': [],
}

def transform_tag(tag):
    out = transform_tag_mapping[tag]
    out = [tag] + out
    out = "|".join(out)
    return out

df["newTag"] = df.Tag.apply(transform_tag)

df.head(30)

,Sentence #,Word,POS,Tag,newTag
0,Sentence: 1,Thousands,NNS,O,O
1,NaN,of,IN,O,O
2,NaN,demonstrators,NNS,O,O
3,NaN,have,VBP,O,O
4,NaN,marched,VBN,O,O
5,NaN,through,IN,O,O
6,NaN,London,NNP,B-geo,B-geo|Location|Party
7,NaN,to,TO,O,O
8,NaN,protest,VB,O,O
9,NaN,the,DT,O,O


In [3]:
from label_functions.basicTypes import is_Float, is_Integer
from label_functions.CountryCode import is_CountryCode
from label_functions.CryptoCurrencyCode import is_CryptoCurrencyCode
from label_functions.CurrencyCode import is_CurrencyCode
from label_functions.TemporalUnit import is_TemporalUnit
from label_functions.Timezone import is_TimeZone
from label_functions.US_States import is_US_States
from label_functions.isMonth import isMonth

In [12]:
text = "usd"

In [13]:
is_CurrencyCode(text)

True

In [14]:
is_Integer("1,234")

True

In [15]:
class label_function:
    def __init__(self, function, tags = [], seperator = "|"):
        self.label_function = function
        self.tags = seperator.join(tags)
        self.seperator = seperator
    def label(self, text, origin_label):
        if self.label_function(text):
            return f"{origin_label}{self.seperator}{self.tags}"
        else:
            return origin_label

In [4]:
labels = [
    [is_Float, ["Float"]],
    [is_Integer, ["Integer"]],
    [is_CountryCode, ["CountryCode"]],
    [is_CryptoCurrencyCode, ["CryptoCurrencyCode"]],
    [is_CurrencyCode, ["CurrencyCode"]],
    [is_TemporalUnit, ["TemporalUnit"]],
    [is_TimeZone, ["Timezone"]],
    [is_US_States, ["US_States"]],
    [isMonth, ['TemporalUnit', 'Month']]
]

In [5]:
label_functions = []
for label in labels:
    label_functions.append(label_function(label[0], label[1]))

NameError: name 'label_function' is not defined

In [18]:
def label_with_functions(row):
    for function in label_functions:
        row["newTag"] = function.label(row.Word, row.newTag)
    return row

In [19]:
import swifter

In [20]:
%%time
df2 = df.swifter.apply(label_with_functions, axis = 1)

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

CPU times: user 7.11 s, sys: 0 ns, total: 7.11 s
Wall time: 1min 30s


In [21]:
#remove O if it have other value
df2.newTag = df2.newTag.apply(lambda x: x[2:] if "O|" == x[0:2] else x)

In [22]:
def remove_repeat_tag(text):
    return "|".join(list(set(text.split("|"))))

In [23]:
df2.newTag = df2.newTag.swifter.apply(remove_repeat_tag)

Pandas Apply:   0%|          | 0/1048575 [00:00<?, ?it/s]

In [24]:
np.unique(df2.newTag)

array(['B-art|CountryCode|Object', 'B-art|Integer|Object', 'B-art|Object',
       'B-tim|Time', 'B-tim|Time|CurrencyCode', 'B-tim|Time|Float',
       'B-tim|Time|Integer', 'B-tim|Time|Month|TemporalUnit',
       'B-tim|Time|TemporalUnit', 'B-tim|Time|Timezone', 'CountryCode',
       'CryptoCurrencyCode', 'CurrencyCode', 'Event|B-eve',
       'Event|CurrencyCode|B-eve', 'Event|Integer|B-eve', 'Float',
       'I-art|Float|Object', 'I-art|Integer|Object', 'I-art|Object',
       'I-eve|CurrencyCode|Event', 'I-eve|Event', 'I-gpe|Race|Party',
       'Integer', 'Location|Party|B-geo',
       'Location|Party|B-geo|CountryCode', 'Location|Party|B-geo|Float',
       'Location|Party|B-geo|Timezone',
       'Location|Party|CurrencyCode|B-geo',
       'Location|Party|CurrencyCode|I-geo', 'Location|Party|I-geo',
       'Location|Party|I-geo|CountryCode',
       'Location|Party|I-geo|Timezone', 'Location|Party|Integer|I-geo',
       'Location|Party|Month|I-geo|TemporalUnit',
       'Location|Party|US

In [25]:
df2.to_csv("./NER_multilabel_data_v2.csv")